## Stock History Generator

This notebook will run similar to the Stock Generator App, except this notebook generates a history of data as quickly as it can between the start/end dates specified. By default, the app will simulate a new reading every 1 second, but only record the result every 1 minute. When complete, results will be written to the specified table (created if necessary). Data will overwrite existing symbol/timestamps (default). 

Each full day takes approx 6 seconds to generate.

In [ ]:
target_table = "raw_stock_test"

startdatetime = "2023-06-01 00:00:00"
enddatetime = "2023-06-04 23:59:00"

In [ ]:
import time
import os
import datetime
import json
import math
import random

from datetime import timedelta
from pyspark import SparkFiles
from pyspark.sql.functions import col
from enum import Enum


In [ ]:
# for parameter preparation, get min/max in current target table
if spark.catalog.tableExists(target_table):
    df = spark.sql(f"SELECT min(timestamp) as mindate, max(timestamp) as maxdate FROM {target_table}")
    df.show()

In [ ]:
end_timestamp = datetime.datetime.strptime(enddatetime, '%Y-%m-%d %H:%M:%S')
timestamp = datetime.datetime.strptime(startdatetime, '%Y-%m-%d %H:%M:%S')

end_timestamp = end_timestamp.replace(second=0, microsecond=0)
timestamp = timestamp.replace(second=0, microsecond=0)

intervalInSeconds = 1 # seconds to add each interval/tick, default 1

# how often to record values, 60 = once per minute
writeFrequency = 60
# how often to print status - default once per day of data
statusFrequency = 86400

sleepTime = 0 # seconds to sleep between intervals/ticks, default 1

In [ ]:
def create_raw_table_if_needed():
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {target_table} (
            symbol VARCHAR(5)
            ,price DOUBLE
            ,timestamp TIMESTAMP
            )
        USING DELTA
        """)

In [ ]:
# merge the predicitions with the table in the lakehouse

from delta.tables import *

def write_rawstock(df, overwrite = False):

    raw_stock_data = DeltaTable.forName(spark, target_table)

    raw_stock_data.alias('raw') \
    .merge( \
        df.alias('StockData'),
        'raw.timestamp = StockData.timestamp and raw.symbol = StockData.symbol'
    ) \
    .whenMatchedUpdate(set =
        {
            "price": f"CASE WHEN '{str(overwrite)}' == 'True' THEN StockData.Price ELSE raw.price END"
        }
    ) \
    .whenNotMatchedInsert(values =
        {
            "symbol": "StockData.symbol"
            ,"price": "StockData.price"
            ,"timestamp": "StockData.timestamp"
        }
    ) \
    .execute()


## Generator Parameters

The 3 main parameters are:

<symbol>_vars: each stock (| (pipe) delimited) values

Events: Occasional triggers that causes all stocks to rise/fall

Timers: Follows time-based rules and influences the behavior of specified stocks

In [ ]:
# # cell is frozen as these are baseline parameters as used in the container. 
# # recommend keeping these as-is for reference: copy and edit as desired

# # each stock can be configured using the pipe-delimited values 
# # the values are:
# # starting price | min price | max price | mu | sigma | correction chance | correction length | correction modifier |
# # 0-20 increase chance | 20-40 increase chance | 40-60 increase chance | 60-80 increase chance | 80-100 increase chance | annual growth rate

# WHO_vars="600|100|1200|.04|0.9|0.01|60|0.4|0.510|0.505|0.500|0.484|0.442|0.08"
# WHAT_vars="500|50|1050|.04|0.8|0.01|60|0.4|0.510|0.502|0.500|0.481|0.442|0.07"
# IDK_vars="500|100|1100|.04|0.9|0.01|60|0.4|0.535|0.540|0.520|0.500|0.475|0.065"
# WHY_vars="550|25|1200|.04|0.9|0.01|60|0.4|0.515|0.515|0.503|0.480|0.442|-0.02"
# BCUZ_vars="300|5|950|.03|0.7|0.01|60|0.4|0.520|0.510|0.505|0.500|0.465|0.06"
# TMRW_vars="500|50|1100|.07|1.0|0.01|60|0.4|0.530|0.520|0.515|0.502|0.430|0.052"
# TDY_vars="700|225|1250|.07|1.0|0.01|60|0.4|0.530|0.520|0.515|0.502|0.430|0.02"
# IDGD_vars="500|50|1050|.04|0.8|0.01|60|0.4|0.503|0.500|0.496|0.492|0.451|0.055"

# # events are occasional triggers that move all stocks in a direction for a set of time
# EventsJson = '{"events": [{"type": "periodic", "name": "900-up", "frequency":900, "increasechance":1.0, "duration": 60, "modifier": 0.5},{"type": "periodic", "name": "5220-down", "frequency":5220, "increasechance":0.0, "duration": 30, "modifier": 0.5},{"type": "random", "name": "Rando1", "frequency": 0.003, "increasechance": 0.504, "duration": 30, "modifier": 0.4}]}'

# # timers are occasional triggers that are time/date based and can be applied to specific stocks
# TimersJson = '{"timers": [{"name": "Workdays", "start":"08:00:00", "end":"18:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.0004, "appliedTo": "WHO|WHAT|IDK|WHY|BCUZ|TMRW|TDY|IDGD"}, {"name": "Evening Decline", "start":"22:00:00", "end":"23:59:59", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.03, "appliedTo": "WHO|WHAT|IDK|WHY|IDGD"}, {"name": "Morning Rise", "start":"04:00:00", "end":"06:00:00", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.0003, "appliedTo": "WHO|WHAT|IDK|WHY|BCUZ|TMRW|TDY|IDGD"}, {"name": "ET Business Hours MWF", "start":"14:00:00", "end":"22:00:00", "days":"0|2|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.04, "appliedTo": "WHO|WHAT|WHY"}, {"name": "WHO Fridays", "start":"14:00:00", "end":"22:00:00", "days":"4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.06, "appliedTo": "WHO"}, {"name": "GMT Business Hours M-F", "start":"08:00:00", "end":"17:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.015, "appliedTo": "TMRW|TDY"},  {"name": "Weekend Slide", "start":"00:00:00", "end":"23:59:59", "days":"5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.012, "appliedTo": "TMRW|TDY"}, {"name": "GMT Business Hours M", "start":"07:00:00", "end":"18:00:00", "days":"0", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.025, "appliedTo": "TMRW|TDY"}, {"name": "Lunch Slump", "start":"12:00:00", "end":"13:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.01, "appliedTo": "WHO|WHAT|IDK|WHY|IDGD"}, {"name": "Hour of Darkness", "start":"01:00:00", "end":"02:00:00", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.055, "appliedTo": "IDGD"}, {"name": "Happy Wednesdays", "start":"01:00:00", "end":"23:00:00", "days":"2", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.035, "appliedTo": "IDGD"}, {"name": "BCUZ Weekends", "start":"00:00:00", "end":"23:59:59", "days":"5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.026, "appliedTo": "BCUZ"}]}'

# printOnlyErrors = True
# maxErrorCount = 15
# useGrowthRate = True
# growthInceptionDate = datetime.datetime.strptime("2023-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')



In [ ]:
# each stock can be configured using the pipe-delimited values 
# the values are:
# starting price | min price | max price | mu | sigma | correction chance | correction length | correction modifier |
# 0-20 increase chance | 20-40 increase chance | 40-60 increase chance | 60-80 increase chance | 80-100 increase chance | annual growth rate

WHO_vars="600|100|1200|.04|0.9|0.01|60|0.4|0.510|0.505|0.500|0.484|0.442|0.08"
WHAT_vars="500|50|1050|.04|0.8|0.01|60|0.4|0.510|0.502|0.500|0.481|0.442|0.07"
IDK_vars="500|100|1100|.04|0.9|0.01|60|0.4|0.535|0.540|0.520|0.500|0.475|0.065"
WHY_vars="550|25|1200|.04|0.9|0.01|60|0.4|0.515|0.515|0.503|0.480|0.442|-0.02"
BCUZ_vars="300|5|950|.03|0.7|0.01|60|0.4|0.520|0.510|0.505|0.500|0.465|0.06"
TMRW_vars="500|50|1100|.07|1.0|0.01|60|0.4|0.530|0.520|0.515|0.502|0.430|0.052"
TDY_vars="700|225|1250|.07|1.0|0.01|60|0.4|0.530|0.520|0.515|0.502|0.430|0.02"
IDGD_vars="500|50|1050|.04|0.8|0.01|60|0.4|0.503|0.500|0.496|0.492|0.451|0.055"

# events are occasional triggers that move all stocks in a direction for a set of time
EventsJson = '{"events": [{"type": "periodic", "name": "900-up", "frequency":900, "increasechance":1.0, "duration": 60, "modifier": 0.5},{"type": "periodic", "name": "5220-down", "frequency":5220, "increasechance":0.0, "duration": 30, "modifier": 0.5},{"type": "random", "name": "Rando1", "frequency": 0.003, "increasechance": 0.504, "duration": 30, "modifier": 0.4}]}'

# timers are occasional triggers that are time/date based and can be applied to specific stocks
TimersJson = '{"timers": [{"name": "Workdays", "start":"08:00:00", "end":"18:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.0004, "appliedTo": "WHO|WHAT|IDK|WHY|BCUZ|TMRW|TDY|IDGD"}, {"name": "Evening Decline", "start":"22:00:00", "end":"23:59:59", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.03, "appliedTo": "WHO|WHAT|IDK|WHY|IDGD"}, {"name": "Morning Rise", "start":"04:00:00", "end":"06:00:00", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.0003, "appliedTo": "WHO|WHAT|IDK|WHY|BCUZ|TMRW|TDY|IDGD"}, {"name": "ET Business Hours MWF", "start":"14:00:00", "end":"22:00:00", "days":"0|2|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.04, "appliedTo": "WHO|WHAT|WHY"}, {"name": "WHO Fridays", "start":"14:00:00", "end":"22:00:00", "days":"4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.06, "appliedTo": "WHO"}, {"name": "GMT Business Hours M-F", "start":"08:00:00", "end":"17:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.015, "appliedTo": "TMRW|TDY"},  {"name": "Weekend Slide", "start":"00:00:00", "end":"23:59:59", "days":"5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.012, "appliedTo": "TMRW|TDY"}, {"name": "GMT Business Hours M", "start":"07:00:00", "end":"18:00:00", "days":"0", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.025, "appliedTo": "TMRW|TDY"}, {"name": "Lunch Slump", "start":"12:00:00", "end":"13:00:00", "days":"0|1|2|3|4", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.01, "appliedTo": "WHO|WHAT|IDK|WHY|IDGD"}, {"name": "Hour of Darkness", "start":"01:00:00", "end":"02:00:00", "days":"0|1|2|3|4|5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":-0.055, "appliedTo": "IDGD"}, {"name": "Happy Wednesdays", "start":"01:00:00", "end":"23:00:00", "days":"2", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.035, "appliedTo": "IDGD"}, {"name": "BCUZ Weekends", "start":"00:00:00", "end":"23:59:59", "days":"5|6", "months": "1|2|3|4|5|6|7|8|9|10|11|12", "modifier":0.026, "appliedTo": "BCUZ"}]}'

printOnlyErrors = True
maxErrorCount = 15
useGrowthRate = True
growthInceptionDate = datetime.datetime.strptime("2023-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')


In [ ]:
class StockVariables:
    def __init__(self, stockVariables) -> None:
        self.startPrice = float(stockVariables[0])
        self.minPrice = float(stockVariables[1])
        self.maxPrice = float(stockVariables[2])
        self.currentPrice = float(stockVariables[0])
        self.mu = float(stockVariables[3])
        self.sigma = float(stockVariables[4])
        self.correctionChance = float(stockVariables[5])
        self.correctionLength = int(stockVariables[6])
        self.correctionCounter = 0
        self.correctionModifier = float(stockVariables[7])
        self.isInCorrection = False
        self.isCorrectionUpwards = True
        self.aboveStartingCount = 1
        self.belowStartingCount = 1
        self.moveUpCount = 1
        self.moveDownCount = 1
        self.increaseChance_0_20 = float(stockVariables[8])
        self.increaseChance_20_40 = float(stockVariables[9])
        self.increaseChance_40_60 = float(stockVariables[10])
        self.increaseChance_60_80 = float(stockVariables[11])
        self.increaseChance_80_100 = float(stockVariables[12])
        self.growthRateAnnual = float(stockVariables[13])
        self.growthRateDaily = self.growthRateAnnual / 365
    
    def getMaxPrice(self):
        if useGrowthRate and self.growthRateAnnual != 0:
            # get days since inception
            daysSinceInception = (timestamp - growthInceptionDate).days
            # calculate growth rate
            growthRateModifier = math.pow(1 + self.growthRateDaily, daysSinceInception)
            # apply growth rate
            newMaxPrice = self.maxPrice * growthRateModifier
            # make sure new max price is greater than min price
            if newMaxPrice < (self.minPrice + 1):
                self.growthRateDaily = abs(self.growthRateDaily)
                newMaxPrice = self.minPrice + 1
                
            return newMaxPrice
        else:
            return self.maxPrice
    
    def getPriceRange(self):
        return self.currentPrice / (self.getMaxPrice() - self.minPrice)
    
    def getIncreaseChance(self, timerModifier = 0.0):

        r = self.getPriceRange()
        increaseChance = self.increaseChance_0_20
        if (r >= 0.80): increaseChance = self.increaseChance_80_100
        elif (r >= 0.60): increaseChance = self.increaseChance_60_80
        elif (r >= 0.40): increaseChance = self.increaseChance_40_60
        elif (r >= 0.20): increaseChance = self.increaseChance_20_40

        increaseChance = increaseChance + timerModifier
        if increaseChance < 0.0: increaseChance = 0.0
        if increaseChance > 1.0: increaseChance = 1.0
        
        return increaseChance

In [ ]:
dataTable = [
     ['WHO', StockVariables(WHO_vars.split('|'))] 
    ,['WHAT', StockVariables(WHAT_vars.split('|'))] 
    ,['IDK', StockVariables(IDK_vars.split('|'))] 
    ,['WHY', StockVariables(WHY_vars.split('|'))] 
    ,['BCUZ', StockVariables(BCUZ_vars.split('|'))] 
    ,['TMRW', StockVariables(TMRW_vars.split('|'))] 
    ,['TDY', StockVariables(TDY_vars.split('|'))] 
    ,['IDGD', StockVariables(IDGD_vars.split('|'))] 
    ]

class MessageType(Enum):
    INFO = 1
    ERROR = 2

def printMsg(message, messageType = MessageType.INFO):
    if (printOnlyErrors == False) or (printOnlyErrors and messageType == MessageType.ERROR):
        print(message)

In [ ]:
numEvents = 0
numTimers = 0

try:
    AllEvents = json.loads(EventsJson)
    numEvents = len(AllEvents['events'])
except Exception as e:
    numEvents = 0
    print(f"{datetime.datetime.utcnow()} Error parsing Events JSON: {e}")
    raise e

if numEvents > 0:
    for event in AllEvents['events']:
        event['durationCount'] = event['duration']
        if (event['type'] == 'periodic'):
            event['frequencyCount'] = event['frequency'] 
        print(f'Event: {event["name"]} {str(event["type"])} {str(event["frequency"])}' \
            f'{str(event["duration"])} {str(event["increasechance"])}')
try:
    AllTimers = json.loads(TimersJson)
    numTimers = len(AllTimers['timers'])

    if numTimers > 0:
        for timer in AllTimers['timers']:
            timer['start'] = datetime.datetime.strptime(timer['start'], "%H:%M:%S").time()
            timer['end'] = datetime.datetime.strptime(timer['end'], "%H:%M:%S").time()
            print(f'Timer: {timer["name"]} {str(timer["start"])} {str(timer["end"])}' \
                f'{str(timer["days"])} {str(timer["months"])} {str(timer["appliedTo"])} {str(timer["modifier"])}')   
            
except Exception as e:
    numTimers = 0
    print(f"{datetime.datetime.utcnow()} Error parsing Timers JSON: {e}")
    raise e


In [ ]:
isEvent = False
currentEvent = ""
count = 0
errorCount = 0

oldtimestamp = timestamp - timedelta(days=1)
readings = []

print("Starting generator...")
rountine_start_time = datetime.datetime.utcnow()
interval_start_time = datetime.datetime.utcnow()

while True:

    if numEvents > 0 and isEvent == False:
        for event in AllEvents['events']:
            if (event['type'] == 'periodic' and event['frequencyCount'] <= 0):
                # periodic event triggered
                event['frequencyCount'] = event['frequency']
                event['durationCount'] = event['duration']
                event['isIncreasing'] = random.random() < event['increasechance']
                currentEvent = event
                isEvent = True
                printMsg(f'{event["name"]} Event Triggered ({"UP" if event["isIncreasing"] else "DOWN"})')
                break
            elif (event['type'] == 'random' and random.random() < event['frequency']):
                # random event triggered
                event['durationCount'] = event['duration']
                event['isIncreasing'] = random.random() < event['increasechance']
                currentEvent = event
                isEvent = True
                printMsg(f'{event["name"]} Event Triggered ({"UP" if event["isIncreasing"] else "DOWN"})')
                break

    for record in dataTable:

        symbol = record[0]
        stockVariables = record[1]
        price = stockVariables.currentPrice

        # apply timers
        # modifier is cumulative across all timers
        currentTimerModifier = 0.0
        appliedTimers = 0
        if numTimers > 0:
            for timer in AllTimers['timers']:
                if (timer['start'] <= timestamp.time() <= timer['end'] 
                        and symbol in timer['appliedTo'].split('|')
                        and str(timestamp.weekday()) in timer['days'].split('|')
                        and str(timestamp.month) in timer['months'].split('|')):
                    currentTimerModifier += timer['modifier']
                    appliedTimers += 1

        # priceIncDec = abs(price - (random.normalvariate(stockVariables.mu, stockVariables.sigma) * price))
        priceIncDec = abs(round(random.normalvariate(stockVariables.mu, stockVariables.sigma),2))

        priceIncrease = random.random() < stockVariables.getIncreaseChance(currentTimerModifier)

        if isEvent:
            if currentEvent['durationCount'] <= 0:
                isEvent = False
            else:
                priceIncrease = currentEvent['isIncreasing'] # force direction if in correction
                priceIncDec = priceIncDec * currentEvent['modifier'] # make corrections more gradual
            stockVariables.isInCorrection = False # force individual corrections off if event

        else:
            if stockVariables.isInCorrection == False and random.random() < stockVariables.correctionChance:
                stockVariables.isInCorrection = True
                stockVariables.correctionCounter = stockVariables.correctionLength
                stockVariables.isCorrectionUpwards = random.random() < stockVariables.getIncreaseChance(currentTimerModifier)

        if stockVariables.isInCorrection:
            if stockVariables.correctionCounter <= 0:
                stockVariables.isInCorrection = False
            else:
                priceIncrease = stockVariables.isCorrectionUpwards # force direction if in correction
                priceIncDec = priceIncDec * stockVariables.correctionModifier # make corrections more gradual
                stockVariables.correctionCounter -= 1

        if priceIncrease:
            newPrice = round(price + priceIncDec,2)
            newPrice = round(newPrice if newPrice < stockVariables.getMaxPrice() else stockVariables.getMaxPrice(),2)
            stockVariables.moveUpCount += 1
            #increase
        else:
            newPrice = round(price - priceIncDec,2)
            newPrice = (newPrice if newPrice > stockVariables.minPrice else stockVariables.minPrice)
            stockVariables.moveDownCount += 1
            #decrease

        stockVariables.currentPrice = newPrice
        if (stockVariables.currentPrice > stockVariables.startPrice):
            stockVariables.aboveStartingCount += 1
        else:
            stockVariables.belowStartingCount += 1
        
        record[1] = stockVariables
        
        reading = {'symbol': symbol, 'price': newPrice, 'timestamp': str(timestamp)}

        if count % writeFrequency == 0:
            readings.append(reading)
             
    if count % statusFrequency == 0:
            interval_finish_time = datetime.datetime.utcnow()
            interval_elap = interval_finish_time - interval_start_time
            s = json.dumps(reading)
            print(f"{datetime.datetime.utcnow()} ({interval_elap.total_seconds()}s): {s}")
            interval_start_time = datetime.datetime.utcnow()

    count += 1

    if isEvent:
        currentEvent["durationCount"] -= 1
    
    if numEvents > 0:
        for event in AllEvents['events']:
            if (event['type'] == 'periodic'):
                event['frequencyCount'] -= 1
        
    oldtimestamp = timestamp
    timestamp = timestamp + datetime.timedelta(seconds=intervalInSeconds)
    if (timestamp > end_timestamp):
        break
    
    if sleepTime>0:
        time.sleep(sleepTime)


## Process all readings

Load readings into dataframe, save to table or CSV as desired

In [ ]:
# load the readings list into a dataframe

readings_df = spark.read.json(sc.parallelize(readings))

In [ ]:
display(readings_df)

In [ ]:
# will create destination table if not exists
create_raw_table_if_needed()

# merge the data
write_rawstock(readings_df, True)


In [ ]:
# optionally write to csv file 

# CSV_FILE_PATH = f"Files/stockhistory/manual/run_name"
# readings_df.write.option("header","true").csv(CSV_FILE_PATH)

# read csv back to dataframe

# df_stocks = (
#     spark.read.format("csv")
#     .option("header", "true")
#     .load(f"Files/stockhistory/manual/run_name/*.csv")
# )

# display(df_stocks)

In [ ]:
# utility - file cleanup if necessary

import os, shutil

def deleteFolder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    os.rmdir(folder)

# path = "/lakehouse/default/Files/stockhistory/manual/"
# deleteFolder(path)

In [ ]:
# utility statements - use with caution!

# spark.sql(f"DELETE FROM {target_table}")
# spark.sql(f"DROP TABLE {target_table}")

In [ ]:
spark.sql(f"SELECT min(timestamp) as mindate, max(timestamp) as maxdate FROM {target_table}").show()
spark.sql(f"SELECT * FROM {target_table} ORDER BY timestamp DESC LIMIT 20").show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

readings_df_pd = readings_df.toPandas()
symbols_pd = sorted(readings_df_pd['symbol'].unique())

fig = go.Figure()

for symbol in symbols_pd:
    # print(symbol)
    dftemp = readings_df_pd.loc[readings_df_pd['symbol'] == symbol][["timestamp","price"]]
    dftemp = dftemp.set_index(pd.DatetimeIndex(dftemp["timestamp"])).drop("timestamp", axis=1)

    # use resample when graphing to limit data points on graph
    # dftemp = dftemp.resample("D").mean()
    # dftemp = dftemp.resample("5min").mean()
    dftemp = dftemp.resample("2H").mean()
 
    dftemp.reset_index(inplace = True)

    fig.add_trace(go.Scatter(x=dftemp['timestamp'], y=dftemp['price'], name=symbol, line=dict(width=1)))

fig.update_layout(title="Generated Data", showlegend=True)
fig.show()